In [1]:
from pyspark.sql import SparkSession, functions as F

# Create a spark session
spark = (
    SparkSession.builder.appName("BNPL External Data-set")
    .config("spark.sql.repl.eagerEval.enabled", True)
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.driver.memory", "4g")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .getOrCreate()
)

22/09/08 21:55:02 WARN Utils: Your hostname, DESKTOP-85B961I resolves to a loopback address: 127.0.1.1; using 172.17.194.172 instead (on interface eth0)
22/09/08 21:55:02 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/08 21:55:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
# Load income by SA2 df
income_by_sa2 = spark.read.parquet('../../data/tables/income_by_sa2.parquet')

In [4]:
transactions = spark.read.parquet('../../data/curated/process_data.parquet')

AnalysisException: Path does not exist: file:/mnt/d/.2022sem2/ADS/.assignments/working_dir/generic-buy-now-pay-later-project-group-56-bnlp/data/curated/process_data.parquet

In [3]:
income_by_sa2.select('SA2').distinct().count()
#income_by_sa2.columns

2288

Only getting 2288 distinct SA2 district's from the ABS data set -> some must be missing?

In [5]:
# read in postcode linking file
postcodes = spark.read.options(header=True).csv('../../data/tables/postcode_sa2_linking_file.csv')

In [6]:
print(postcodes.count())
postcodes_feature_subset = ['postcode', 'SA2_MAINCODE_2016', 'locality', 'lat', 'long', 'electoraterating']
postcodes = postcodes.select(*postcodes_feature_subset)

18442


In [7]:
postcodes.limit(3)

postcode,SA2_MAINCODE_2016,locality,lat,long,electoraterating
0200,801051049,ANU,-35.2777,149.119,null
0200,801051049,Australian Nation...,-35.2777,149.1189,null
0800,701011002,DARWIN,-12.458684,130.83668,Inner Metropolitan


Make a subset just for linking + piece of mind

In [8]:
link_set =  ['postcode', 'SA2_MAINCODE_2016']
link = postcodes.select(*link_set)
link.limit(3)

postcode,SA2_MAINCODE_2016
0200,801051049
0200,801051049
0800,701011002


In [9]:
# descriptive statistics
print("Distinct postcodes: ", link.select('postcode').distinct().count())
print("Distinct SA2 regions: ", link.select('SA2_MAINCODE_2016').distinct().count())

Distinct postcodes:  3167
Distinct SA2 regions:  2222


Again -> this is less than the officially reported number of ~ 2310, might have to look into using geopandas and closest centroids...

#### Take the income dataset; match sa2 to postcodes; aggregate across postcodes for final set

In [31]:
link.printSchema()

root
 |-- postcode: string (nullable = true)
 |-- SA2_MAINCODE_2016: string (nullable = true)



In [32]:
income_by_sa2.printSchema()

root
 |-- SA2: string (nullable = true)
 |-- SA2_NAME: string (nullable = true)
 |-- persons_earners_2014-15: string (nullable = true)
 |-- persons_earners_2015-16: string (nullable = true)
 |-- persons_earners_2016-17: string (nullable = true)
 |-- persons_earners_2017-18: string (nullable = true)
 |-- persons_earners_2018-19: string (nullable = true)
 |-- med_age_earners_2014-15: string (nullable = true)
 |-- med_age_earners_2015-16: string (nullable = true)
 |-- med_age_earners_2016-17: string (nullable = true)
 |-- med_age_earners_2017-18: string (nullable = true)
 |-- med_age_earners_2018-19: string (nullable = true)
 |-- sum_earnings_2014-15: string (nullable = true)
 |-- sum_earnings_2015-16: string (nullable = true)
 |-- sum_earnings_2016-17: string (nullable = true)
 |-- sum_earnings_2017-18: string (nullable = true)
 |-- sum_earnings_2018-19: string (nullable = true)
 |-- median_earnings_2014-15: string (nullable = true)
 |-- median_earnings_2015-16: string (nullable = true)


In [13]:
income_by_sa2.limit(3)

22/09/08 17:18:46 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


SA2,SA2_NAME,persons_earners_2014-15,persons_earners_2015-16,persons_earners_2016-17,persons_earners_2017-18,persons_earners_2018-19,med_age_earners_2014-15,med_age_earners_2015-16,med_age_earners_2016-17,med_age_earners_2017-18,med_age_earners_2018-19,sum_earnings_2014-15,sum_earnings_2015-16,sum_earnings_2016-17,sum_earnings_2017-18,sum_earnings_2018-19,median_earnings_2014-15,median_earnings_2015-16,median_earnings_2016-17,median_earnings_2017-18,median_earnings_2018-19,mean_earnings_2014-15,mean_earnings_2015-16,mean_earnings_2016-17,mean_earnings_2017-18,mean_earnings_2018-19
101021007,Braidwood,2133,2153,2262,2315,2361,50,50,50,51,51,101830606,109961530,115564780,119439615,120763285,38093,39716,41288,42003,41593,47741,51074,51090,51594,51149
101021008,Karabar,4866,4937,4988,5059,5100,43,42,42,42,42,289762288,301618607,311923113,322715297,338308979,54942,55870,57880,59295,61777,59548,61093,62535,63790,66335
101021009,Queanbeyan,6194,6419,6486,6595,6697,40,39,39,39,39,367366063,390044004,395374979,414579067,441160946,54716,54999,55376,57848,60119,59310,60764,60958,62863,65874


In [14]:
link.limit(3)

postcode,SA2_MAINCODE_2016
0200,801051049
0200,801051049
0800,701011002


In [167]:
inc_joined = link.join(income_by_sa2, link.sa2_code == income_by_sa2.SA2, "inner").distinct()
#inc_joined = link.join(income_by_sa2, link.SA2_MAINCODE_2016 == income_by_sa2.SA2, "left").distinct()
print(inc_joined.count())
inc_joined.select(['postcode', 'sa2_code']).orderBy('postcode').limit(5)
#inc_joined = inc_joined.drop('SA2_MAINCODE_2016')

# end up with 5488 rows
# therefore there a lot of postcodes with SA2's > 1 || SA2's with postcodes > 1

5488


postcode,sa2_code
0200,801051049
0800,701011002
0801,701011002
0804,701011007
0810,701021013


In [170]:
inc_joined.select('SA2').distinct().count()
#inc_joined.select('postcode').distinct().count()

2217

In [17]:
from pyspark.sql import functions as F

In [18]:
inc_cols = inc_joined.columns

In [19]:
inc_joined.select('postcode').distinct().count()

3167

In [171]:
agg_by_postcode_income = inc_joined \
    .groupBy('postcode') \
    .agg(
    F.sum('persons_earners_2018-19').alias('persons_earners_2018-19_sum'),
    F.mean('mean_earnings_2018-19').alias('mean_earnings_2018-19_avg'),
    F.sum('sum_earnings_2018-19').alias('sum_earnings_2018-19_sum'),
    F.mean('median_earnings_2018-19').alias('median_earnings_2018-19_avg'),
    F.mean('med_age_earners_2018-19').alias('med_age_earners_2018-19_avg')
    #F.percentile_approx('median_earnings_2018-19', 0.5).alias('median_earnings_2018-19_med'),
    #F.percentile_approx('med_age_earners_2018-19', 0.5).alias('med_age_earners_2018-19_med')
) \
    .orderBy('postcode')

In [172]:
agg_by_postcode_income.show()

+--------+---------------------------+-------------------------+------------------------+---------------------------+---------------------------+
|postcode|persons_earners_2018-19_sum|mean_earnings_2018-19_avg|sum_earnings_2018-19_sum|median_earnings_2018-19_avg|med_age_earners_2018-19_avg|
+--------+---------------------------+-------------------------+------------------------+---------------------------+---------------------------+
|    0200|                      552.0|                  19479.0|             1.0752338E7|                    10433.0|                       23.0|
|    0800|                     5632.0|                  74682.0|            4.20609031E8|                    57789.0|                       33.0|
|    0801|                     5632.0|                  74682.0|            4.20609031E8|                    57789.0|                       33.0|
|    0804|                     1810.0|                  88303.0|            1.59828824E8|                    71724.0|       

In [40]:
import sys
sys.path.append('../../scripts/')

In [43]:
#import etl
#from etl import preprocess_population

ModuleNotFoundError: No module named 'nltk'

In [98]:
# Anuj population code from ETL
population = spark.read.option("header", True).csv('../../data/tables/population_data.csv')

In [99]:
from pyspark.sql.types import LongType, IntegerType
def population_preprocess(data):

    cols_to_keep = ['sa2_maincode_2016', 'sa2_name_2016', 'erp_2021']
    population_df = data.select(*cols_to_keep)

    population_df = population_df \
                    .withColumn("sa2_maincode_2016", F.col("sa2_maincode_2016").cast(LongType())) \
                    .withColumn("erp_2021", F.col('erp_2021').cast(IntegerType()))

    population_df = population_df \
                    .withColumnRenamed('sa2_name_2016', 'suburb') \
                    .withColumnRenamed('erp_2021', 'estimated_region_population_2021')

    return population_df

In [100]:
population = population_preprocess(population)

In [101]:
print("Before linking rows (population): " , population.count())
print("Before linking rows (postcode links): " , link.count())
#population.head(5)
population.limit(5)

Before linking rows (population):  2292
Before linking rows (postcode links):  18442


sa2_maincode_2016,suburb,estimated_region_population_2021
101021007,Braidwood,4360
101021008,Karabar,8374
101021009,Queanbeyan,11401
101031013,Bombala,2411
101041020,Bega-Eden Hinterland,8996


In [69]:
link.limit(5)

postcode,SA2_MAINCODE_2016
0200,801051049
0200,801051049
0800,701011002
0800,701011002
0801,701011002


In [144]:
link = link.distinct()
link = link.withColumnRenamed('SA2_MAINCODE_2016', 'sa2_code')
link.orderBy('postcode').limit(5)

postcode,sa2_code
0200,801051049
0800,701011002
0801,701011002
0804,701011007
0810,701021013


In [145]:
# investigate distincts
print("Distinct SA2 Codes: (link)", link.select('sa2_code').distinct().count())
print("Distinct postcodes (link): ", link.select('postcode').distinct().count())
print("Distinct entries: (link)", link.distinct().count())

print("Distinct SA2 Codes: (population)", population.select('sa2_maincode_2016').distinct().count())
#print("Distinct postcodes (link): ", link.select('postcode').distinct().count())
#print("Distinct entries: (link)", link.select('postcode').distinct().count())


Distinct SA2 Codes: (link) 2222
Distinct postcodes (link):  3167
Distinct entries: (link) 5588
Distinct SA2 Codes: (population) 2292


In [113]:
population.count()
#population.distinct().count()

2292

In [151]:
#population_joined = population.join(link, population['sa2_maincode_2016'] == link['SA2_MAINCODE_2016'], "inner")
population_joined = population.join(link, population['sa2_maincode_2016'] == link['sa2_code'], "inner").distinct()

In [155]:
print(population_joined.count())
population_joined.orderBy('postcode').orderBy('postcode').limit(20)

5492


sa2_maincode_2016,suburb,estimated_region_population_2021,postcode,sa2_code
801051049,Acton,2161,0200,801051049
701011002,Darwin City,7088,0800,701011002
701011002,Darwin City,7088,0801,701011002
701011007,Parap,2827,0804,701011007
701021021,Lyons (NT),6892,0810,701021021
701021023,Millner,2427,0810,701021023
701021024,Moil,1997,0810,701021024
701021013,Brinkin - Nakara,3154,0810,701021013
701021016,Coconut Grove,3064,0810,701021016
701021010,Alawa,2081,0810,701021010


In [156]:
print("distinct postcodes = ", population_joined.select('postcode').distinct().count())
print("distinct sa2_codes = ", population_joined.select('sa2_code').distinct().count())

#population_joined.na.drop().count()

distinct postcodes =  3165
distinct sa2_codes =  2221


In [157]:
agg_by_postcode_population = population_joined \
        .groupBy('postcode') \
        .agg(
        F.sum('estimated_region_population_2021').alias('estimated_region_population_2021_sum'),
        ) \
        .orderBy('postcode')

In [160]:
agg_by_postcode_population.limit(10)

postcode,estimated_region_population_2021_sum
0200,2161
0800,7088
0801,7088
0804,2827
0810,35007
0811,3154
0812,18442
0813,2848
0814,3719
0815,3154
